# C/C++ 101

**Table of contents:**
 - [C](#C)
 - [C++](#sec-cpp)
 - [Summary](#Summary)

If you have never written a program in C/C++, this notebook provides the basic concepts to follow along.

## C

<div class="alert alert-block alert-success">
    <ul>
        <li>C is a simple and small language</li>
        <li>C is a DIY language (write your own data structures!)</li>
        <li>C is a generic assembly language</li>
        <li>If there's anything "missing" from C it's λ-functions</li>
    </ul>
</div>

One of the simplest C programs is shown below. It includes a module `stdio.h` that provides standard I/O functions including `printf` that lets programs print a string to a terminal.

In [ ]:
%%writefile src/hello.c
#include <stdio.h>

int main()
{
    printf("Hello, World!\n");
    return 0;
}

A C program (source code) must be compiled to machine code before it can be executed.

In [ ]:
%%bash
gcc -o build/hello src/hello.c
build/hello

The HDF5 library appears to a C program as another module that provides functions to perform HDF5 tasks. Using the HDF5 library, in the example below, we retrieve and print the HDF5 library version, and create an "empty" HDF5 file.

Unlike several other languages, a variable definition in C must indicate the variable's type, which precedes the variable name. E.g., the variables `[maj,min,rel]num` are of type `unsigned integer`.

External modules typically introduce API-specific types. Perhaps the most important API-type in the HDF5 library is `hid_t`. It is used to identify and track entities managed by the HDF5 library.

Since the C runtime doesn't have a garbage collector, the programmer is in charge of managing (and not wasting) resources. Similarly, the HDF5 library delegates control over HDF5 entities (identified by an `hid_t`!) to the programmer. Perhaps the single most important rule when using the HDF5 C-API in a program is Powell's rule: 

<div class="alert alert-block alert-warning">
<b>The Powell rule:</b> <i>"If you break it, you are going to own it."</i> In other words, if you call an HDF5 library API (e.g., <tt>H5*[create,open]</tt>) that returns a value of type <tt>hid_t</tt>, then you are going to own the underlying resources allocated by the HDF5 library. You must explicitely release these resources by calling appropriate functions (e.g., <tt>H5*close</tt>) that tell the library that the application does no longer require those resources. Failing to release unused resources (a resource leak) often leads to poor performance, abnormal program behavior, or application or system crashes, etc.
</div>

 Case in point: See the matching `H5Fcreate` and `H5Fclose` calls in the example below.

In [ ]:
%%writefile src/hello_hdf5.c
#include "hdf5.h"
#include <stdio.h>

int main(int argc, char **argv)
{
    unsigned majnum, minnum, relnum;
    H5get_libversion(&majnum, &minnum, &relnum);
    printf("Hello, HDF5 library %d.%d.%d!", majnum, minnum, relnum);

    // If the user provided a filename, create a file under that name
    // otherwise, create a file called "build/hello.h5"
    hid_t file = H5Fcreate((argc > 1 ? argv[1] : "build/hello.h5"), H5F_ACC_TRUNC, H5P_DEFAULT, H5P_DEFAULT);
    H5Fclose(file);

    return 0;
}

Since the HDF5 library is not part of the standard set of modules that ships with the C compiler, we must tell the compiler (and linker) something about the location of the HDF5 module via command line options. (See the compiler manual for details: `man gcc`)

In [ ]:
%%bash
gcc -o build/hello_hdf5 src/hello_hdf5.c -Wall -pedantic -I/usr/include/hdf5/serial -L/usr/lib/x86_64-linux-gnu -lhdf5_serial
build/hello_hdf5 build/foo.h5

<a id="sec-cpp"></a>
## C++

<div class="alert alert-block alert-success">
    <ul>
        <li>C++ is not a small language (but it's not a strict superset of C!)</li>
        <li>C++ comes with batteries included</li>
        <li>C++ has namespaces (and modules as of C++20)</li>
        <li>C++ has λ-functions (since C++11)</li>
        <li>When used competently, in modern C++, you can get the conciseness of Python <i>and</i> the performance of C</li>
    </ul>
</div>

There are several (C++) idiomatic language bindings available for the HDF5 library, including __[HighFive](https://bluebrain.github.io/HighFive/poster/)__, __[H5CPP](https://github.com/steven-varga/h5cpp)__, and __[h5cpp](https://github.com/ess-dmsc/h5cpp)__. Programmers who want their code to look like modern C++ (rather than C) should check them out! The Powell rule still applies, but can be conveniently folded into idioms such as __[RAII](https://en.cppreference.com/w/cpp/language/raii)__.

In this tutorial, we take advantage of a few batteries included with C++ while sticking with the HDF5 library C-API. That way, our examples are less verbose than plain C, but it's easier to follow how the HDF5 libary API is used. On the flipside, we won't be able to showcase the elegance of C++, but that isn't a goal of this tutorial.

A C++ compiler would have no issue with the C example shown earlier, but a C compiler would not be satisfied with the C++ code shown next.

In [ ]:
%%writefile src/hello_hdf5.cpp
#include "hdf5.h"
#include <iostream>

int main(int argc, char **argv)
{
    unsigned majnum, minnum, relnum;
    H5get_libversion(&majnum, &minnum, &relnum);
    std::cout << "Hello, HDF5 library "
        << majnum << "." << minnum << "." << relnum << "!" << std::endl;

    // If the user provided a filename, create a file under that name
    // otherwise, create a file called "build/hello.h5"
    hid_t file = H5Fcreate((argc > 1 ? argv[1] : "build/hello.1.h5"), H5F_ACC_TRUNC, H5P_DEFAULT, H5P_DEFAULT);
    H5Fclose(file);

    return 0;
}

In [ ]:
%%bash
g++ -o build/hello_hdf5.1 src/hello_hdf5.cpp -Wall -pedantic -I/usr/include/hdf5/serial -L/usr/lib/x86_64-linux-gnu -lhdf5_serial
build/hello_hdf5.1 build/bar.h5

## Summary

C/C++ programs need to be compiled into machine code before they can be executed. Variables are typed and the most important HDF5 library C-API type is `hid_t` for identifying and tracking HDF5 entities. Values of type `hid_t` are signs which symbolize resources allocated by the HDF5 library on behalf of the caller. The caller is responsible for releasing resources no longer required (Powell rule). For application developers, several fine C++ language bindings of the HDF5 library are available. If you are already using the HDF5 library through __[`h5py`](https://www.h5py.org/)__, __[HDFql](https://www.hdfql.com/)__, or __[`rhdf5`](https://bioconductor.org/packages/release/bioc/html/rhdf5.html)__, good for you! &#128077;